In [ ]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

'''
Plot forecast maps with all available models.
'''

%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import numpy.ma as ma
import pandas as pd
import struct
import os
import xarray as xr
import glob
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from esio import EsioData as ed
from esio import ice_plot
from esio import import_data
from esio import metrics
import dask

# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=.8, rc={"lines.linewidth": 2.5})

In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=2)
# client = Client()
# client
dask.config.set(scheduler='threads')  # overwrite default with threaded scheduler
# dask.config.set(scheduler='processes')  # overwrite default with threaded scheduler

In [ ]:
# Parameters
pred_year = 2018 # Prediction year


In [ ]:
#############################################################
# Load in Data
#############################################################

E = ed.EsioData.load()
mod_dir = E.model_dir

# Get recent observations
ds_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True).sic
#ds_51 = xr.open_mfdataset(E.obs['NSIDC_0051']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)
ds_79 = xr.open_mfdataset(E.obs['NSIDC_0079']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True).sic

In [ ]:
# Fill in pole hole with sic of 1
ds_81 = ds_81.where(ds_81.hole_mask==0, other=1)
ds_79 = ds_79.where(ds_79.hole_mask==0, other=1)

In [ ]:
ds_81.time[0].values, ds_81.time[-1].values

In [ ]:
ds_79.time[0].values, ds_79.time[-1].values

In [ ]:
# Combine bootstrap with NASA NRT
da_sic = ds_79.combine_first(ds_81)

In [ ]:
# Reshape as year x doy
# Add DOY
DOY_all = [x.timetuple().tm_yday for x in pd.to_datetime(da_sic.time.values)]
da_sic.coords['doy'] = xr.DataArray(DOY_all, dims='time', coords={'time':da_sic.time})
year_all = [x.year for x in pd.to_datetime(da_sic.time.values)]
da_sic.coords['year'] = xr.DataArray(year_all, dims='time', coords={'time':da_sic.time})

In [ ]:
# Grab sic dataArray
da_sic.load()

# Climatology forecast

In [ ]:
test_plots = False

In [ ]:
cmod = 'climatology'
runType = 'forecast'
Last_N_years = 'all' # Number of years prior to pred_year to train on. Use 'all' to use all available.

if Last_N_years=='all':
    Last_N_years = (pred_year-da_sic.year.min()).item()
    

for cdoy in np.arange(1,da_sic.doy.max().values+1,1):
    print("Processing day of year",cdoy)

    # Select current day of year
    da_cdoy = da_sic.where(da_sic.doy==cdoy, drop=True).swap_dims({'time':'year'})
    
    # Split by train and validate years
    da_train = da_cdoy.sel(year=slice(pred_year - Last_N_years, pred_year - 1))
    
    # Call model
    ds_pred = metrics.linearRegressionModel(da_train.chunk({'year': -1}), 'year', pred_year) # Have to rechunk year into one big one
    
    # Store
    ds_pred.coords['doy'] = cdoy
    
    # Force prediction SIC to be between 0-1
    ocnmask = ds_pred.notnull()
    ds_pred = ds_pred.where(ds_pred >= 0, other=0).where(ocnmask)
    ds_pred = ds_pred.where(ds_pred <= 1, other=1).where(ocnmask)
    
    if test_plots:
        xr.exit()
    
    # Move back to actual (valid_time) space
    ds_pred = ds_pred.expand_dims('time')
    ds_pred.coords['time'] = xr.DataArray([datetime.datetime(pred_year,1,1) + datetime.timedelta(days=int(x-1)) for x in [cdoy]], dims='time')
        
    ds_pred.name = 'sic'

    if not test_plots:
        # Save to disk
        file_out = os.path.join(mod_dir, cmod, runType, 'sipn_nc', str(pred_year)+'_'+format(cdoy, '03')+'_'+str(pred_year - Last_N_years)+'_'+str(pred_year - 1)+'_SIC.nc')
        ds_pred.to_netcdf(file_out)
        print("Saved",file_out)
    

# Clim trend last 10 years

### For presentation figures

In [ ]:
if test_plots:
    fig_dir = '/home/disk/sipn/nicway/Nic/figures/pres/A'

    cx = 160
    cy = 220

    sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})
    f = plt.figure()
    da_train.isel(year=1).T.plot(label='Sea ice Concentration')
    plt.plot(cy,cx,marker='o',markersize=10,color='k')
    plt.title('')
    f_out = os.path.join(fig_dir,'spatial_plot.png')
    f.savefig(f_out,bbox_inches='tight', dpi=300)

    sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})





    from scipy import stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(da_train.isel(y=cy,x=cx).year.values, da_train.isel(y=cy,x=cx).values)
    predict_y = intercept + slope * da_train.isel(y=cy,x=cx).year.values
    predict_y

    f = plt.figure()
    da_train.isel(y=cy,x=cx).plot(color='b',label='Observed')
    plt.plot(2018, ds_pred.isel(y=cy,x=cx).values,'r*',label='Predicted',markersize=14)
    plt.plot(da_train.isel(y=cy,x=cx).year.values, predict_y,'k--', label='linear least-squares')
    plt.title('')
    plt.legend(loc='lower left', bbox_to_anchor=(1.03, .7))
    plt.ylabel('Sea Ice Concentration (-)')
    f_out = os.path.join(fig_dir,'linearfit.png')
    f.savefig(f_out,bbox_inches='tight', dpi=300)